In [1]:
from nltk.metrics import *
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import pprint

In [2]:
#https://www.dataquest.io/blog/web-scraping-tutorial-python/ used as a resource for this HW
from bs4 import BeautifulSoup
import requests

In [3]:
# Read in spreadsheet with links
df = pd.read_excel(r'C:\Users\seans\Desktop\NLP\HW5\permalinks.xlsx', header=None)

#convert to list to iterate through
link_list = df[0].tolist()

Inspect the html to see where our reviews live

In [4]:
#Downloads the page
page = requests.get("https://www.imdb.com/review/rw5020019/?ref_=tt_urv")

In [5]:
#Raw content
page.content

b'\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/ns#"\n    xmlns:fb="http://www.facebook.com/2008/fbml">\n    <head>\n         \n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///review/rw5020019?src=mdot">\n\n\n\n        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:\'java\'};</script>\n\n<script>\n    if (typeof uet == \'function\') {\n      uet("bb", "LoadTitle", {wb: 1});\n    }\n</script>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>\n        <title>iGlad\'s Review of The Boys - IMDb</title>\n  <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>\n<script>\n    if (typeof uet == \'function\') {\n      uet("be", "LoadTitle", {wb: 1});\n    }\n</script>\n<script>\n    if

In [6]:
#Structure the html so we can find the review text tag
soup = BeautifulSoup(page.content, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="app-id=342792525, app-argument=imdb:///review/rw5020019?src=mdot" name="apple-itunes-app"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   iGlad's Review of The Boys - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld", "LoadTitle

In [7]:
list(soup.children)

['\n',
 'html',
 '\n',
 <html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
 <meta charset="utf-8"/>
 <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
 <meta content="app-id=342792525, app-argument=imdb:///review/rw5020019?src=mdot" name="apple-itunes-app"/>
 <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
 <script>
     if (typeof uet == 'function') {
       uet("bb", "LoadTitle", {wb: 1});
     }
 </script>
 <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
 <title>iGlad's Review of The Boys - IMDb</title>
 <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
 <script>
     if (typeof uet == 'function') {
       uet("be", "LoadTitle", {wb: 1});
     }
 </script>
 <script>
     if (typeof uex == 'function') {
       uex("ld", "LoadTitle", {wb: 1});
     }
 

In [8]:
[type(item) for item in list(soup.children)]

[bs4.element.NavigableString,
 bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [9]:
html = list(soup.children)[3]

In [10]:
html

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///review/rw5020019?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>iGlad's Review of The Boys - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/revi

#### Within the html, the review text lives in the 'div' with the 'text show-more__control' class.  

In [11]:
span = str(soup.find('div', attrs={'text show-more__control'}).text).strip()

In [12]:
span

"Having being all superheroed out with the never ending train of films this is so much welcome relief. It's not for kids and I love the choice language which is appropriate."

In [192]:
#NP chunker taken from: https://www.nltk.org/book/ch07.html

#Remove punctuation
punc= ".,`''\\"
import string

#Put everything in lowercase
span = span.lower()

#Remove punctuation for stemmer
for i in span:
    for j in punc:
        if i in punc:
            span = span.replace(i, "")
            
#First we need to tokenize and tag the sentence
tokenized = word_tokenize(span)
tagged = nltk.pos_tag(tokenized)

#Define that we want a determinant, aand a noun potentially preceeded by an adjective
np_rule = "NP: {<DT>?<JJ.*>*<NN.*>+}"
np = nltk.RegexpParser(np_rule)

np_chunked = np.parse(tagged)




In [193]:
print(np_chunked)

(S
  having/VBG
  being/VBG
  all/DT
  superheroed/VBN
  out/RP
  with/IN
  the/DT
  never/RB
  ending/VBG
  (NP train/NN)
  of/IN
  (NP films/NNS)
  this/DT
  is/VBZ
  so/RB
  (NP much/JJ welcome/JJ relief/NN)
  its/PRP$
  not/RB
  for/IN
  (NP kids/NNS)
  and/CC
  i/VB
  love/VBP
  (NP the/DT choice/NN language/NN)
  which/WDT
  is/VBZ
  appropriate/JJ)


### Chuck all reviews and add to the list

In [194]:
output = {}
counter = 1

np_rule = "NP: {<DT>?<JJ.*>*<NN.*>+}"
np = nltk.RegexpParser(np_rule)

for i in link_list:
    lst = []
    page = requests.get(i)
    soup = BeautifulSoup(page.content, 'html.parser')
    span = str(soup.find('div', attrs={'text show-more__control'}).text).strip()
    tokenized = word_tokenize(span)
    tagged = nltk.pos_tag(tokenized)
    chunked = np.parse(tagged)
    for subtree in chunked.subtrees():
        if subtree.label() == 'NP':
            lst.append(subtree)
    
    output["Review: " + str(counter)] = lst
    counter +=1

In [195]:
output

{'Review: 1': [Tree('NP', [('this', 'DT'), ('brilliant', 'JJ'), ('spin', 'NN')]),
  Tree('NP', [('a', 'DT'), ('superhero', 'NN'), ('show', 'NN')]),
  Tree('NP', [('dinner', 'NN')]),
  Tree('NP', [('the', 'DT'), ('whole', 'NN')]),
  Tree('NP', [('first', 'JJ'), ('episode', 'NN')]),
  Tree('NP', [('this', 'DT'), ('exciting', 'NN')]),
  Tree('NP', [('crazy', 'JJ'), ('ride', 'NN')])],
 'Review: 2': [Tree('NP', [('train', 'NN')]),
  Tree('NP', [('films', 'NNS')]),
  Tree('NP', [('much', 'JJ'), ('welcome', 'JJ'), ('relief', 'NN')]),
  Tree('NP', [('kids', 'NNS')]),
  Tree('NP', [('the', 'DT'), ('choice', 'NN'), ('language', 'NN')])],
 'Review: 3': [Tree('NP', [('Excellent', 'JJ'), ('dystopian', 'JJ'), ('reimagining', 'NN')]),
  Tree('NP', [('Superhero', 'NNP'), ('nerds', 'NNS')]),
  Tree('NP', [('Surprising', 'NNP')]),
  Tree('NP', [('violent', 'NN')]),
  Tree('NP', [('Binge-watched', 'NNP')]),
  Tree('NP', [('days', 'NNS')]),
  Tree('NP', [('a', 'DT'), ('second', 'JJ'), ('season', 'NN')]),


# TextBlob Parser and NP Extraction

In [18]:
#https://textblob.readthedocs.io/en/dev/quickstart.html#noun-phrase-extraction
from textblob import TextBlob
blobbed = TextBlob(span)
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\seans\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.


True

In [26]:
blobbed.noun_phrases[0]

'no excuse'

In [27]:
output = {}
counter = 1

for i in link_list:
    lst = []
    page = requests.get(i)
    soup = BeautifulSoup(page.content, 'html.parser')
    span = str(soup.find('div', attrs={'text show-more__control'}).text).strip()
    blobbed = TextBlob(span)
    lst.append(blobbed.noun_phrases[0:])
    #for subtree in chunked.subtrees():
        #if subtree.label() == 'NP':
            #lst.append(subtree)
    
    output["Review: " + str(counter)] = lst
    counter +=1

In [30]:
[print("{}{}\n\n".format(key,value)) for key, value in output.items()]

Review: 1[WordList(['started', 'brilliant spin', 'superhero show', 'whole thing.from'])]


Review: 2[WordList(['choice language'])]


Review: 3[WordList(['excellent', 'superhero', 'surprising', 'binge-watched', 'thumbs'])]


Review: 4[WordList(['whole superhero genre', 'great actors', 'great storyline', 'nice picture', 'superhero series', 'titans', 'umbrella'])]


Review: 5[WordList(['around', 'global obsession', 'comic characters', 'dollar industry', 'real life', 'easily', 'superhero', 'tv show', 'marvels daredevil', 'top violence', 'great dialogues', 'kiwi', 'lead protagonists', 'kurl urban', 'anthony starr', 'starr', 'lead antagonist', 'homelander', 'superhero movie', 'was', 'wow', 'amazon prime', 'production cost', 'watch'])]


Review: 6[WordList(['ca', 'anthony', 'aussie', 'homelander', 'ca'])]


Review: 7[WordList([])]


Review: 8[WordList(['season', 'dark knight', 'mcu', "'s funnier", 'excellent satire', 'american/western', 'neo-feudal corporatocracy', 'corporatist media', 'actu

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]